In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Transfer Learning with TensorFlow: Feature Extraction**

To improve models we can use several tenchniques like
- Adding More Layers
- Changing the Learning Rate
- Adjusting the Number of Neurons Per Layer

However, instead of above we can use **Transfer Learning**.
- **Transfer Learning** is taking the patterns (also called weights) from another model and applying it on new problem.

Two main benefits of using Transfer Learning.
- Leverage existing Neural Network Architecture proven to work on problems similar to our own
- Using **Already Learned Patterns** on similar data to our own

So, instead of hand-crafting our own **Neural Network Architecture or building them from scratch** we can utilize models which have worked for others.

We can take the patterns a model has learned from datasets such as **ImageNet** and use it as a foundational model.

**What we will Learn**
- Use a smaller dataset to experiment faster (10% of training samples of 10 classes of food)
- Build a Transfer Learning Feature Extraction model using **TensorFlow Hub**
- Introduce a TesnorBoard Callback to track model training results

**Transfer Learning with TensorFlow Hub: Getting great results with only 10% of data**

- **TensorFlow Hub:-** is a repository for existing model components. You can import and use a **Fully Trained Model** using a *URL*

Using the **Pre-trained Models** we can get the results of a fully trained model with only 10% of data.

**Transfer Learning often allows you to get great results with less data**


Let's download 10% of training data from **10_food_classes** dataset and use it to train a food image classifier on it.















In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2025-09-22 08:34:59--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.250.101.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   213MB/s    in 0.8s    

2025-09-22 08:35:00 (213 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
import zipfile
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [4]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ch

**Creating Data Loaders (Preparing the Data)**

- Create the **ImageDataGenerator** class using the **flow_from_directory** method to load in our images.

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training Images")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical"
                                                          )


print("Testing Images")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical"
                                             )



Training Images
Found 750 images belonging to 10 classes.
Testing Images
Found 2500 images belonging to 10 classes.


**Setting Up Callbacks (Things to Run While our Model Trains)**

**Callbacks** are extra functionality that you can add to your models to be performed during or after training. Some of the most important callbacks are

- **Experiment Tracking with TensorBoard:-** Log the performance of multiple models and then view and compare these models in a visual way on **TensorBoard**. **TensorBoard** is a dashboard for inspecting **Neural Network Parameters**

- **Model CheckPointing:-** Save your model as you train so that you can stop training if needed and continue off where you left. It is helpful if training takes a long time and cannot be done in one sitting.

- **Early Stopping:-** Leave your model training for a arbitary amount of time and have it stop training automatically when it ceases to improve. It is helpful when you have a large dataset and do not know how long training will take.

- The TensorBoard Callback can be accessed using
**tf.keras.callbacks.TensorBoard()**. - The main function of this is saving model's training performance metrics to a specified **log_dir**.

- By default, logs are recorded every epoch using the **update_freq='epoch'** parameter. This is a good default but can slow down **Model Training.**





In [6]:
# Creating a tensorboard callback

import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback


- We will save the **Model** to a directory [dir_name] / [experiment_name] / [current_timestamp] where
- **dir_name:-** is the overall logs directory
- **experiment_name:-** is the particular experiment
- **current_timestamp:-** is the time the experiment started based on Python time *datatime.datetime().now()

**Creating Models Using TensorFlow Hub**

- In past we used to create models from scratch
- In here, majority of our model's layers are going to come from **TensorFlow Hub**

We will use two models from **TensorFlow Hub**

- 1.**ResNetV2** - a state of the art computer vision model architecture from 2016
- 2.**EfficientNet**- a state of the art computer vision model from 2019

By the state of art we mean that majority of our modeles have achieved the lowest error rate on **ImageNet (ILSVRC-2012-CLS)**, the gold standard of computer vision benchmarks.

Steps for finding models on **TensorFlow Hub**
1. Got to **tfhub.dev**
2. Select the problem domain like **Image**
3. Remove all **Problem Domain** filters except for the one you are working on
4. You will see a list of models, select the one you want to use

**I see many models, then which one is to be used**

- You can find a list of state of the art models on **paperswithcode.com**, a resource for collecting the latest in deep learning paper results

- Since, our target is **Image Classification** so we would use the model that performed best on **ImageNet.**

- On **tfhub.dev** you will find various architectures like **EfficientNetB4** which is better than **EfficientNetB0** but larger models take alot of time to compute.


We will use feature vectors URLs of two common computer vision architectures, **EfficientNetBO (2019)** and **ResNetV250 (2016)**

Why we select only **Feature Vectors**.
Because Transfer Learning come into play as **Feature Extraction** and **Fine Tuning**

1. In **transfer Learning** we take a pre-trained models as it is and apply it to our task without changes. For example, if your model is trained on **ImageNet** dataset that contains **1000** different classes of images. So, if we pass a single image to this model it will produce **1000** different outputs. It can be useful if we want to classify **1000** images.

2. **Feature Extraction Transfer Learning** is a process where you take the underlying patterns (also called weights) a **pretrained Model** has learned and adjust its output to be more suited to your problem.
For example, If your model had 236 different layers (EfficientNetBO has 236 layers) and the top layer outputs **1000** classes because it was pretrained on **ImageNet.**To adjust it to your problem we might remove the top layer and replace it with our own having the right number of classes. The most important part here is that **only the top few layers become trainable, the rest remain frozen.**So, the underlying patterns remain in the rest of layers and we can utilize it for our problem.

3. **Fine-Tuning Transfer Learning:-** is when you take the underlying patterns (also called weights) of a pre-trained model and adjust them to your problem. This means **training some, many or all layers** in pretrained model. This is applicable in scenarios where you have relatively large dataset and **your data is slightly different** from the original data on which the model was trained.


A common practice is to **Freeze** all the learned patterns in bottom layers of a **pretrained model** so that they become **un-trainable**. Then, the top 2-3 layers of the **pre-trained** model can adjust its output to our customer data (**feature extraction**).
As you have trained the **top layers** you can gradually **unfreeze** more and more layers and run the training process on your own data to further **fine-tune**it.

- **Lower Layers** in a computer vision model learns **large features**. In a cat and dog classification they might learn the **outline of legs** while the layers closer to the output might learn **shape of the teeth.**


So, in **Feature Extraction** only the top 2-3 layers change but in **Fine Tuning Model** many or all of the original model get changed.





In [7]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [8]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"


- The **URLs** are link to a saved **pretrained models** on **TensorFlow Hub**.
- When we use it in our model, the model will automatically downloaded for us to use
- We will use **KerasLayer()** inside the TensorFlow Hub library.

- The function below helps in creating **Model**.
- Our first model will be **ResNetV250** architecture as our feature extraction layer
- Once our model is instantiated, we will compile it using **categorical_crossentropy** as our loss function, **Adam Optimizer** and **Accuracy** as metric.

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

# Define constants
IMAGE_SHAPE = (224, 224)
NUM_CLASSES = 10  # Replace with the actual number of classes in your dataset
RESNET_URL = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Custom layer to wrap hub.KerasLayer
class HubLayer(tf.keras.layers.Layer):
    def __init__(self, model_url, **kwargs):
        super(HubLayer, self).__init__(**kwargs)
        self.hub_layer = hub.KerasLayer(model_url, trainable=False)

    def call(self, inputs):
        return self.hub_layer(inputs)

def create_model_tf_hub_fixed(model_url, num_classes=NUM_CLASSES):
    # Define input layer
    inputs = keras.Input(shape=IMAGE_SHAPE + (3,))

    # Apply custom Hub layer
    x = HubLayer(model_url)(inputs)

    # Add dropout and dense layers
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = keras.Model(inputs, outputs)

    return model

# Clear Keras session
tf.keras.backend.clear_session()

# Disable mixed precision
tf.keras.mixed_precision.set_global_policy('float32')

# Create and compile the model
resnet_model = create_model_tf_hub_fixed(RESNET_URL, num_classes=NUM_CLASSES)
resnet_model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

# Display model summary
resnet_model.summary()




Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hub_layer (HubLayer)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,490 (80.04 KB)

 Trainable params: 20,490 (80.04 KB)

 Non-trainable params: 0 (0.00 B)

**Adding the CallBack**
- A tensorflow **CallBack** adds extra functionality by virtue of which we can track the performance of our model on TensorBoard
- The **CallBack** is added in the **fit** method

In [ ]:
resnet_history = resnet_model.fit(
    train_data_10_percent,
    epochs=5,
    steps_per_epoch=len(train_data_10_percent),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                           experiment_name="resnet50V2")]
)


Saving TensorBoard log files to: tensorflow_hub/resnet50V2/20250922-084601
Epoch 1/5
 1/24 ━━━━━━━━━━━━━━━━━━━━ 23:15 61s/step - accuracy: 0.0625 - loss: 3.2295